In [1]:
from sklearn.linear_model import LinearRegression
import numpy as np
import scipy.stats as stats
from matplotlib.pyplot import plot
import pandas as pd
import pylab
import statsmodels.api as sm 
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_predict, cross_val_score
import statsmodels.formula.api as smf
from sklearn.metrics import mean_squared_error, r2_score

In [2]:
df = pd.read_excel('data/HW_Data_Set.xlsx')

In [3]:
df.describe()

,ind_5,ind_6,ind_8,ind_9,ind_10,ind_12,ind_13,ind_14,ind_15,ind_16,...,ind_412,ind_414,ind_416,ind_418,ind_424,ind_426,ind_428,20_target,50_target,90_target
count,6167.000000,6167.000000,6167.000000,6167.000000,6167.000000,6167.000000,6167.000000,6167.000000,6167.000000,6167.000000,...,6167.000000,6167.000000,6167.000000,6167.000000,6167.000000,6167.000000,6167.000000,6167.000000,6167.000000,6167.000000
mean,-0.803146,-0.803470,48.388890,48.289282,0.099609,49.488867,-18.497518,11.771485,11.773550,11.773392,...,0.000224,28.529204,-8.554792,-28.166694,0.490297,0.498188,0.490174,-0.057102,-2.906358,-3.315072
std,23.624403,23.624144,36.388526,36.478009,60.414625,12.198722,68.281120,6.803997,5.786089,4.970499,...,0.005660,26.316896,16.100087,23.271524,0.425376,0.228547,0.150483,14.834497,34.559778,36.841541
min,-131.000000,-131.000000,0.000000,0.000000,-100.000000,12.134540,-625.093855,1.050000,1.660000,2.580000,...,-0.023790,0.000000,-103.800000,-188.000000,0.000000,0.000000,0.083530,-15.780588,-37.142889,-38.573858
25%,-14.000000,-14.000000,14.285714,14.285714,-50.000000,40.868503,-45.687212,6.850000,7.260000,7.635000,...,-0.003230,10.700000,-11.700000,-39.000000,0.000000,0.323783,0.379894,-15.350354,-36.208094,-37.695273
50%,0.000000,0.000000,50.000000,50.000000,0.000000,49.549766,-2.817298,10.550000,11.040000,11.270000,...,-0.000053,21.300000,0.000000,-22.000000,0.473684,0.495192,0.487078,0.000000,-34.267756,-35.788391
75%,13.000000,13.000000,85.714286,85.714286,57.142857,58.504375,28.072613,14.950000,15.040000,14.960000,...,0.003414,38.400000,0.000000,-11.000000,1.000000,0.675730,0.601508,15.385945,35.879524,37.521892
max,76.000000,76.000000,100.000000,100.000000,100.000000,84.821848,81.105847,82.900000,54.240000,37.050000,...,0.023340,281.800000,0.000000,0.000000,1.000000,1.000000,0.868534,15.768767,37.038694,38.465649


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6167 entries, 0 to 6166
Columns: 136 entries, ind_5 to 90_target
dtypes: float64(88), int64(45), object(3)
memory usage: 6.4+ MB


In [5]:
a =  ['ind_420', 'ind_422']
print(a)

['ind_420', 'ind_422']


In [6]:
df1 = df.drop(labels=a, axis=1)

In [7]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6167 entries, 0 to 6166
Columns: 134 entries, ind_5 to 90_target
dtypes: float64(88), int64(45), object(1)
memory usage: 6.3+ MB


In [8]:
cat = {'ind_109':{'GREEN':1,'RED':0}}
df1.replace(cat, inplace=True)

In [9]:
b = list((df1 == 0).astype(int).sum(axis=0).sort_values(ascending=False).head(10).index)
print(b)

['ind_352', 'ind_349', 'ind_351', 'ind_345', 'ind_344', 'ind_347', 'ind_340', 'ind_338', 'ind_334', 'ind_330']


In [10]:
df2 = df1.drop(labels=b, axis=1)
df2.head()

,ind_5,ind_6,ind_8,ind_9,ind_10,ind_12,ind_13,ind_14,ind_15,ind_16,...,ind_412,ind_414,ind_416,ind_418,ind_424,ind_426,ind_428,20_target,50_target,90_target
0,19,17,100.0,85.714286,14.285714,72.363515,60.808814,23.80,17.62,11.73,...,0.002371,2.0,-49.6,-54,1.0,0.498547,0.701906,15.135802,35.625252,36.997753
1,24,19,100.0,78.571429,21.428571,74.275883,64.366798,11.45,18.16,12.22,...,0.003074,0.8,-55.6,-60,1.0,0.537088,0.690833,15.143348,35.643013,37.016198
2,30,24,100.0,71.428571,28.571429,75.140402,65.915803,8.75,17.86,12.28,...,0.003561,2.7,-58.4,-60,1.0,0.615169,0.693040,15.146870,35.651301,-37.024805
3,37,30,100.0,64.285714,35.714286,76.677846,68.584234,7.80,14.76,12.61,...,0.004366,0.9,-61.8,-65,1.0,0.661517,0.673418,15.153283,0.000000,-37.040483
4,41,37,100.0,57.142857,42.857143,81.603007,76.455495,14.90,11.92,14.25,...,0.004198,0.1,-79.8,-86,1.0,0.747204,0.700522,-15.179065,-35.727079,-37.103503


In [11]:
c = ['50_target','20_target']
df3 = df2.drop(labels=c, axis=1)

In [12]:
corre = pd.DataFrame(df3.corr()['90_target'])
corre = corre.transpose()
corre['ind_5'][0]

0.06042800031451863

In [13]:
ll = []
for i in corre:
    if abs(corre[i][0])< 0.04:
        ll.append(i)

In [14]:
corre['ind_132']

90_target   -0.016912
Name: ind_132, dtype: float64

In [15]:
ll

['ind_8',
 'ind_9',
 'ind_10',
 'ind_22',
 'ind_27',
 'ind_28',
 'ind_29',
 'ind_30',
 'ind_31',
 'ind_32',
 'ind_33',
 'ind_39',
 'ind_40',
 'ind_42',
 'ind_43',
 'ind_44',
 'ind_45',
 'ind_46',
 'ind_47',
 'ind_48',
 'ind_51',
 'ind_52',
 'ind_53',
 'ind_56',
 'ind_58',
 'ind_62',
 'ind_65',
 'ind_82',
 'ind_88',
 'ind_92',
 'ind_109',
 'ind_116',
 'ind_122',
 'ind_124',
 'ind_132',
 'ind_134',
 'ind_136',
 'ind_139',
 'ind_145',
 'ind_147',
 'ind_153',
 'ind_156',
 'ind_164',
 'ind_168',
 'ind_172',
 'ind_176',
 'ind_179',
 'ind_181',
 'ind_188',
 'ind_194',
 'ind_212',
 'ind_217',
 'ind_225',
 'ind_234',
 'ind_237',
 'ind_241',
 'ind_243',
 'ind_311',
 'ind_313',
 'ind_327',
 'ind_387',
 'ind_397',
 'ind_399',
 'ind_403',
 'ind_404',
 'ind_407',
 'ind_410',
 'ind_414',
 'ind_424',
 'ind_426']

In [16]:
d = ['ind_412','ind_17','ind_18','ind_109']

In [17]:
X = df3.drop(ll, axis=1)
X = X.drop('90_target',axis=1)
y = df3['90_target']



In [18]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.2, random_state=42)

In [19]:
lm = sm.OLS(y_train,X_train)

In [20]:
model = lm.fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:              90_target   R-squared (uncentered):                   0.119
Model:                            OLS   Adj. R-squared (uncentered):              0.109
Method:                 Least Squares   F-statistic:                              12.88
Date:                Sat, 22 Aug 2020   Prob (F-statistic):                    1.34e-98
Time:                        15:18:32   Log-Likelihood:                         -24502.
No. Observations:                4933   AIC:                                  4.911e+04
Df Residuals:                    4882   BIC:                                  4.944e+04
Df Model:                          51                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
ind_5         -0.0400      0.505     -0.079      0.937      -1.030       0.950
ind_6         -0.0637      0.505     -0.126      0.899      -1.053       0.925
ind_12         1.4887      0.659      2.259      0.024       0.197       2.781
ind_13         0.0028      0.022      0.132      0.895      -0.039       0.045
ind_14         0.1862      0.140      1.325      0.185      -0.089       0.461
ind_15        -0.2012      0.241     -0.835      0.404      -0.673       0.271
ind_16         0.0716      0.375      0.191      0.849      -0.663       0.806
ind_17         0.8433      0.366      2.305      0.021       0.126       1.560
ind_18         1.0457      0.261      4.004      0.000       0.534       1.558
ind_19         0.8695      1.032      0.842      0.400      -1.154       2.893
ind_20        -1.3259      1.034     -1.282      0.200      -3.353       0.702
ind_21         0.9825      1.034      0.950      0.342      -1.046       3.011
ind_23        -0.0730      0.079     -0.923      0.356      -0.228       0.082
ind_25     -1791.4696    743.692     -2.409      0.016   -3249.441    -333.498
ind_26      -794.0936    790.974     -1.004      0.315   -2344.758     756.571
ind_34         0.0331      0.020      1.682      0.093      -0.005       0.072
ind_35        -0.0564      0.022     -2.548      0.011      -0.100      -0.013
ind_36         0.0247      0.017      1.447      0.148      -0.009       0.058
ind_37         0.0439      0.016      2.747      0.006       0.013       0.075
ind_41        -0.0306      0.071     -0.430      0.667      -0.170       0.109
ind_49        -0.0123      0.103     -0.120      0.905      -0.214       0.189
ind_50         0.7225      0.103      7.030      0.000       0.521       0.924
ind_61         0.5655      1.458      0.388      0.698      -2.292       3.423
ind_67        -0.6098      1.711     -0.356      0.722      -3.964       2.745
ind_70         4.7338      1.713      2.763      0.006       1.375       8.093
ind_71         0.0701      0.122      0.576      0.565      -0.169       0.309
ind_74        -0.2055      0.159     -1.289      0.197      -0.518       0.107
ind_77         0.4886      0.133      3.683      0.000       0.229       0.749
ind_83        -0.1543      0.073     -2.128      0.033      -0.296      -0.012
ind_102       -1.9668      0.841     -2.338      0.019      -3.616      -0.318
ind_118     3210.5474   6709.052      0.479      0.632   -9942.214    1.64e+04
ind_128       -2.0815      0.740     -2.811      0.005      -3.533      -0.630
ind_130       -0.6702      0.118     -5.684      0.000      -0.901      -0.439
ind_142        0.8012      0.231      3.466      0.001       0.348       1.254
ind_150       -2.5035      0.792     -3.160      0.002      -

In [21]:
from sklearn import preprocessing
x_scaled = preprocessing.scale(X)

In [37]:
X_train, X_test, y_train, y_test = train_test_split(pd.DataFrame(x_scaled),y,test_size = 0.2, random_state=42)
lm = sm.OLS(y_train,X_train)
model = lm.fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:              90_target   R-squared (uncentered):                   0.113
Model:                            OLS   Adj. R-squared (uncentered):              0.104
Method:                 Least Squares   F-statistic:                              12.18
Date:                Sat, 22 Aug 2020   Prob (F-statistic):                    2.85e-92
Time:                        15:21:44   Log-Likelihood:                         -24518.
No. Observations:                4933   AIC:                                  4.914e+04
Df Residuals:                    4882   BIC:                                  4.947e+04
Df Model:                          51                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
0             -3.0028     11.971     -0.251      0.802     -26.470      20.465
1             -0.2418     11.961     -0.020      0.984     -23.691      23.208
2             29.8053      8.667      3.439      0.001      12.813      46.797
3              3.0793      1.672      1.842      0.066      -0.199       6.357
4              1.3865      0.959      1.446      0.148      -0.493       3.266
5             -1.0322      1.399     -0.738      0.461      -3.775       1.711
6             -0.0208      1.872     -0.011      0.991      -3.690       3.648
7              3.2376      1.654      1.957      0.050      -0.005       6.481
8              2.4979      0.715      3.495      0.000       1.097       3.899
9              7.4716      9.240      0.809      0.419     -10.642      25.586
10           -23.9489     19.744     -1.213      0.225     -62.656      14.758
11            10.4601     11.445      0.914      0.361     -11.978      32.898
12            -1.2886      1.963     -0.657      0.512      -5.137       2.559
13            -2.1811      1.059     -2.059      0.040      -4.257      -0.105
14            -1.4016      1.067     -1.314      0.189      -3.494       0.690
15             5.0720      2.451      2.070      0.039       0.267       9.877
16            -5.8220      2.709     -2.149      0.032     -11.133      -0.511
17             3.9037      2.049      1.905      0.057      -0.114       7.921
18             6.4315      1.930      3.333      0.001       2.648      10.215
19             0.5230      1.520      0.344      0.731      -2.457       3.503
20            -2.1592      2.997     -0.721      0.471      -8.034       3.716
21            30.4226      4.609      6.601      0.000      21.388      39.458
22             4.0374      5.176      0.780      0.435      -6.110      14.185
23            -1.5322      4.421     -0.347      0.729     -10.200       7.135
24            10.3385      3.677      2.811      0.005       3.129      17.547
25             0.5742      1.440      0.399      0.690      -2.248       3.397
26            -2.5701      1.878     -1.369      0.171      -6.252       1.112
27             5.5741      1.560      3.574      0.000       2.516       8.632
28            -1.8965      0.850     -2.232      0.026      -3.563      -0.230
29           -52.9766     20.687     -2.561      0.010     -93.533     -12.420
30            12.5820     13.376      0.941      0.347     -13.641      38.805
31           -70.8357     22.065     -3.210      0.001    -114.093     -27.579
32           -33.1656      5.750     -5.768      0.000     -44.439     -21.893
33             9.2529      2.568      3.603      0.000       4.218      14.288
34           -51.1767     10.724     -4.772      0.000     -7

In [38]:
summe = pd.DataFrame(model.pvalues)


In [39]:
summe = summe.transpose()


In [40]:
ll = []
for i in summe:
    if summe[i][0] > 0.3:
        ll.append(i)

In [41]:
ll

[0, 1, 5, 6, 9, 11, 12, 19, 20, 22, 23, 25, 30, 37, 39, 45, 49]

In [42]:
pd.DataFrame(x_scaled)

,0,1,2,3,4,5,6,7,8,9,...,41,42,43,44,45,46,47,48,49,50
0,0.838318,0.753674,1.875320,1.161562,1.768003,1.010514,-0.008731,0.064130,-0.977360,0.254172,...,0.447570,0.665971,0.091847,0.163974,-1.532911,-1.542396,0.379478,-2.549585,-1.110172,1.407132
1,1.049980,0.838340,2.032100,1.213674,-0.047253,1.103849,0.089859,-0.099256,-0.935807,0.030038,...,0.447655,0.665003,0.091847,0.158695,-1.788981,-1.679542,0.503689,-2.922284,-1.368019,1.333540
2,1.303976,1.050005,2.102976,1.236362,-0.444111,1.051996,0.101931,-0.177979,-0.902565,0.590372,...,0.447741,0.664605,0.091847,0.153500,-1.896075,-1.679542,0.589731,-3.096210,-1.368019,1.348207
3,1.600303,1.304004,2.229019,1.275445,-0.583746,0.516185,0.168328,-0.299776,-0.864706,1.262773,...,0.447827,0.663798,0.091847,0.148386,-2.108731,-1.782401,0.731935,-3.307406,-1.582891,1.217802
4,1.769634,1.600335,2.632796,1.390732,0.459843,0.025313,0.498302,-0.081432,-0.688338,1.486906,...,0.447913,0.660258,0.091847,0.143353,-2.564026,-2.433844,0.702224,-4.425503,-2.485355,1.397934
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6162,1.176978,1.346337,0.862592,0.744924,-0.304476,-0.413707,0.379592,0.741439,1.647853,1.486906,...,0.373355,0.671448,-0.225105,-0.035345,-0.846942,-0.890953,0.779076,-1.220292,-0.508530,0.735506
6163,0.965315,1.177005,0.878945,0.753009,-0.605794,-0.507042,0.003342,0.664202,1.610917,1.262773,...,0.373440,0.671563,-0.225105,-0.046310,-0.844032,-0.822380,1.464356,-1.245138,-0.551504,0.249213
6164,0.838318,0.965339,1.216681,0.908789,-0.414714,-0.469016,-0.296451,0.633010,1.682942,1.262773,...,0.373525,0.670165,-0.225105,-0.056765,-1.217661,-1.062385,1.523932,-1.779340,-0.895300,0.409284
6165,0.838318,0.838340,1.601847,1.063752,0.261414,-0.244321,-0.246151,0.408726,1.722648,1.038639,...,0.373611,0.668200,-0.225105,-0.066745,-1.737617,-1.405250,1.540151,-2.506103,-1.410993,0.673952


In [43]:
x_scaled = pd.DataFrame(x_scaled).drop(ll,axis=1)

y = df3['90_target']
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.2, random_state=42)
lm = sm.OLS(y_train,X_train)
model = lm.fit()
model.summary()


<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:              90_target   R-squared (uncentered):                   0.119
Model:                            OLS   Adj. R-squared (uncentered):              0.109
Method:                 Least Squares   F-statistic:                              12.88
Date:                Sat, 22 Aug 2020   Prob (F-statistic):                    1.34e-98
Time:                        15:21:55   Log-Likelihood:                         -24502.
No. Observations:                4933   AIC:                                  4.911e+04
Df Residuals:                    4882   BIC:                                  4.944e+04
Df Model:                          51                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
ind_5         -0.0400      0.505     -0.079      0.937      -1.030       0.950
ind_6         -0.0637      0.505     -0.126      0.899      -1.053       0.925
ind_12         1.4887      0.659      2.259      0.024       0.197       2.781
ind_13         0.0028      0.022      0.132      0.895      -0.039       0.045
ind_14         0.1862      0.140      1.325      0.185      -0.089       0.461
ind_15        -0.2012      0.241     -0.835      0.404      -0.673       0.271
ind_16         0.0716      0.375      0.191      0.849      -0.663       0.806
ind_17         0.8433      0.366      2.305      0.021       0.126       1.560
ind_18         1.0457      0.261      4.004      0.000       0.534       1.558
ind_19         0.8695      1.032      0.842      0.400      -1.154       2.893
ind_20        -1.3259      1.034     -1.282      0.200      -3.353       0.702
ind_21         0.9825      1.034      0.950      0.342      -1.046       3.011
ind_23        -0.0730      0.079     -0.923      0.356      -0.228       0.082
ind_25     -1791.4696    743.692     -2.409      0.016   -3249.441    -333.498
ind_26      -794.0936    790.974     -1.004      0.315   -2344.758     756.571
ind_34         0.0331      0.020      1.682      0.093      -0.005       0.072
ind_35        -0.0564      0.022     -2.548      0.011      -0.100      -0.013
ind_36         0.0247      0.017      1.447      0.148      -0.009       0.058
ind_37         0.0439      0.016      2.747      0.006       0.013       0.075
ind_41        -0.0306      0.071     -0.430      0.667      -0.170       0.109
ind_49        -0.0123      0.103     -0.120      0.905      -0.214       0.189
ind_50         0.7225      0.103      7.030      0.000       0.521       0.924
ind_61         0.5655      1.458      0.388      0.698      -2.292       3.423
ind_67        -0.6098      1.711     -0.356      0.722      -3.964       2.745
ind_70         4.7338      1.713      2.763      0.006       1.375       8.093
ind_71         0.0701      0.122      0.576      0.565      -0.169       0.309
ind_74        -0.2055      0.159     -1.289      0.197      -0.518       0.107
ind_77         0.4886      0.133      3.683      0.000       0.229       0.749
ind_83        -0.1543      0.073     -2.128      0.033      -0.296      -0.012
ind_102       -1.9668      0.841     -2.338      0.019      -3.616      -0.318
ind_118     3210.5474   6709.052      0.479      0.632   -9942.214    1.64e+04
ind_128       -2.0815      0.740     -2.811      0.005      -3.533      -0.630
ind_130       -0.6702      0.118     -5.684      0.000      -0.901      -0.439
ind_142        0.8012      0.231      3.466      0.001       0.348       1.254
ind_150       -2.5035      0.792     -3.160      0.002      -

In [44]:
a = df3.groupby('ind_109')['90_target'].mean()
a

ind_109
0   -2.583133
1   -4.024343
Name: 90_target, dtype: float64

In [45]:
df3['90_target'].mean()

-3.315072358419006

In [46]:
X = df3[['ind_109']]
y = df3['90_target']
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.2, random_state=42)
lm = sm.OLS(y_train,X_train)
model = lm.fit()
mse = mean_squared_error(y_train,model.fittedvalues)

In [47]:
mse

1361.8887171825982

In [48]:
reg=LinearRegression()
model = reg.fit(X,y)

In [49]:
k_t = pd.DataFrame({'Real':y[0:10],'Pred':reg.predict(X)[0:10]})
k_t

,Real,Pred
0,36.997753,-4.024343
1,37.016198,-4.024343
2,-37.024805,-4.024343
3,-37.040483,-4.024343
4,-37.103503,-4.024343
5,-37.098277,-4.024343
6,-37.096740,-4.024343
7,-37.088440,-4.024343
8,-37.069073,-2.583133
9,-37.074913,-2.583133
